# [문제]
- ChatPromptTemplate() : 역할 부여
- InMemoryChatMessageHistory()
- RunnableWithMessageHistory()
- chain 구성

## [문제] 기존 String PromptTemplate을 ChatPromptTemplate으로 변경
- Chain
- prompt template: 직접 작성
                   문서를 참고해서, 사용자 질문에 답변

In [1]:
## 모듈(파일, 라이브러리) 읽어오기
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from dotenv import load_dotenv

In [2]:
## 2. 환경변수에 설정된 값 읽어오기
load_dotenv()

True

In [3]:
## 3. ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model='gpt-4.1-nano',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

llm

ChatOpenAI(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x0000021489838FD0>], client=<openai.resources.chat.completions.completions.Completions object at 0x000002148C1C77C0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002148C1F2560>, root_client=<openai.OpenAI object at 0x000002148C108EE0>, root_async_client=<openai.AsyncOpenAI object at 0x000002148C1C7880>, model_name='gpt-4.1-nano', model_kwargs={}, openai_api_key=SecretStr('**********'), streaming=True)

In [4]:
## 4. 파일 읽기
file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'
with open(file_name, 'r', encoding='utf-8') as file:
    law = file.read()
    print(law[:50])


남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률 ( 약칭: 남녀고용평등법 )
[시행 20


In [6]:
from langchain_core.prompts import ChatPromptTemplate

template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
     ('placeholder', '{chat_history}'),
    ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)
prompt

ChatPromptTemplate(input_variables=['law', 'query'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessage

In [7]:
query = input('육아휴직과 관련된 질문하세요 >> ')
query

'육아휴직 기간은 어떻게 되나요?'

In [8]:
chain = prompt | llm

In [12]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

history = InMemoryChatMessageHistory()

def get_history():
    return history

with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)

In [13]:
with_message_history.invoke({
    'law': law,
    'query': query
})

육아휴직은 임신 중인 여성 근로자가 모성을 보호하거나 근로자가 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위하여 신청하는 휴직을 말하며, 기간은 원칙적으로 1년 이내입니다. (제19조(육아휴직) ②항)

다만, 다음과 같은 경우에는 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다:
- 같은 자녀를 대상으로 부모가 모두 육아휴직을 각각 3개월 이상 사용한 경우
- 「한부모가족지원법」 제4조제1호의 부 또는 모
- 고용노동부령으로 정하는 장애아동의 부 또는 모 (제19조(육아휴직) ②항)

즉, 일반적으로 육아휴직은 최대 1년까지 가능하며, 특수한 사정이 있는 경우에는 최대 6개월(추가 사용 포함 시 1년)까지 사용할 수 있습니다.

AIMessage(content='육아휴직은 임신 중인 여성 근로자가 모성을 보호하거나 근로자가 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위하여 신청하는 휴직을 말하며, 기간은 원칙적으로 1년 이내입니다. (제19조(육아휴직) ②항)\n\n다만, 다음과 같은 경우에는 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다:\n- 같은 자녀를 대상으로 부모가 모두 육아휴직을 각각 3개월 이상 사용한 경우\n- 「한부모가족지원법」 제4조제1호의 부 또는 모\n- 고용노동부령으로 정하는 장애아동의 부 또는 모 (제19조(육아휴직) ②항)\n\n즉, 일반적으로 육아휴직은 최대 1년까지 가능하며, 특수한 사정이 있는 경우에는 최대 6개월(추가 사용 포함 시 1년)까지 사용할 수 있습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45'}, id='run-9224afce-6bd8-4689-b5c0-024f1c974fda-0')

In [14]:
history

InMemoryChatMessageHistory(messages=[HumanMessage(content='육아휴직 기간은 어떻게 되나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='육아휴직은 임신 중인 여성 근로자가 모성을 보호하거나 근로자가 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하기 위하여 신청하는 휴직을 말하며, 기간은 원칙적으로 1년 이내입니다. (제19조(육아휴직) ②항)\n\n다만, 다음과 같은 경우에는 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다:\n- 같은 자녀를 대상으로 부모가 모두 육아휴직을 각각 3개월 이상 사용한 경우\n- 「한부모가족지원법」 제4조제1호의 부 또는 모\n- 고용노동부령으로 정하는 장애아동의 부 또는 모 (제19조(육아휴직) ②항)\n\n즉, 일반적으로 육아휴직은 최대 1년까지 가능하며, 특수한 사정이 있는 경우에는 최대 6개월(추가 사용 포함 시 1년)까지 사용할 수 있습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45'}, id='run-9224afce-6bd8-4689-b5c0-024f1c974fda-0')])

In [2]:
print('변경!!!')

변경!!!
